In [3]:
%cd t-few

[Errno 2] No such file or directory: 't-few'
/content/drive/MyDrive/drive_workspace/t-few


In [4]:
!pip install omegaconf
!pip install -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ob5ire6d
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-ob5ire6d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [5]:
import os
import torch
import argparse
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from src.data import FinetuneDataModule, get_dataset_reader, PretrainDataModule
from src.models.EncoderDecoder import EncoderDecoder
from src.models.modify_model import modify_transformer
from src.utils.Config import Config
from src.utils.util import ParseKwargs, set_seeds
from omegaconf import OmegaConf
from datasets import load_dataset
import re
from src.models.lora import LoRALinear
import torch.nn as nn
import json


In [6]:



def maybe_modify(config,module,c_name,m_name,layer,debug=False):
    should_modify = bool(re.fullmatch(config.lora_modules, m_name)) and bool(re.fullmatch(config.lora_layers, c_name)) 
                    
    if not debug:
        if should_modify:
            setattr(
                module,
                c_name,
                LoRALinear(layer, config.lora_rank, config.lora_scaling_rank, config.lora_init_scale),
            )
    else:
        return dict(
            module=m_name,
            c_name=c_name,layer=type(layer),should_modify=should_modify,is_linear = isinstance(layer, nn.Linear))

def lora_modify(transformer, config,debug=False):
    d_list = []
    for m_name, module in dict(transformer.named_modules()).items():
        for c_name, layer in dict(module.named_children()).items():
            a = maybe_modify(config,module,c_name,m_name,layer,debug)
            d_list.append(a)

                    
 
    return transformer,d_list

In [7]:
# import pandas as pd
# from transformers import AutoModel

# model_name = "google/t5-v1_1-base"
# model = AutoModel.from_pretrained(model_name)
# config = Config()
# config.update_kwargs(json.loads("""{
#     "lora_scaling_rank": 1,
#     "lora_rank": 0,
#     "lora_init_scale": 0.0,
#     "lora_modules": ".*SelfAttention|.*EncDecAttention|.*DenseReluDense",
#     "lora_layers": "k|v|wi_1.*",
#     "trainable_param_names": ".*lora_b.*",
#     "model_modifier": "lora",
#     "lr": 3e-3,
#     "num_steps": 1000
# }"""),eval=False)

# df = pd.DataFrame(lora_modify(model,config,debug=True)[1])
# df.to_markdown(f"{model_name.split('/')[-1]}.md")

In [8]:

# model_name = "gpt2"
# model = AutoModel.from_pretrained(model_name)
# config = Config()
# config.update_kwargs(json.loads("""{
#     "lora_scaling_rank": 1,
#     "lora_rank": 0,
#     "lora_init_scale": 0.0,
#     "lora_modules": ".*SelfAttention|.*EncDecAttention|.*DenseReluDense",
#     "lora_layers": "k|v|wi_1.*",
#     "trainable_param_names": ".*lora_b.*",
#     "model_modifier": "lora",
#     "lr": 3e-3,
#     "num_steps": 1000
# }"""),eval=False)
# df = pd.DataFrame(lora_modify(model,config,debug=True)[1])
# df.to_markdown(f"{model_name.split('/')[-1]}.md")

In [9]:

dataset = load_dataset("poem_sentiment")
print(dataset)

Using custom data configuration default
Reusing dataset poem_sentiment (/root/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})


In [10]:

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  encoding = tokenizer(batch["verse_text"])
  # For language modeling the labels need to be the input_ids
  #encoding["labels"] = encoding["input_ids"]
  return encoding

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
# The GPT-2 tokenizer does not have a padding token. In order to process the data 
# in batches we set one here 
tokenizer.pad_token = tokenizer.eos_token
column_names = dataset["train"].column_names
dataset = dataset.map(encode_batch, remove_columns=column_names, batched=True)



Parameter 'function'=<function encode_batch at 0x7f780067d3b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
block_size = 50
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
  # Concatenate all texts.
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
  # customize this part to your needs.
  total_length = (total_length // block_size) * block_size
  # Split by chunks of max_len.
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

dataset = dataset.map(group_texts,batched=True,)

dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("facebook/opt-13b")
config  = OmegaConf.create({
    "lora_scaling_rank": 1,
    "lora_rank": 0,
    "lora_init_scale": 0.0,
    "lora_modules": ".*",
    "lora_layers": "k_proj|v_proj|fc2",
    "trainable_param_names": ".*lora_b.*",
    "model_modifier": "lora",
    "optimizer":"adamw",
    "lr": 3e-3,
    "num_steps": 1000,
    "scheduler":"cosine_annealing",
    "warmup_ratio":0.1,
    "weight_decay":0,
    })

model = AutoModelForCausalLM.from_pretrained("facebook/opt-13b")
model,_ = lora_modify(model,config)


: 

: 

In [ ]:
from src.utils.get_optimizer import get_optimizer
from src.utils.get_scheduler import get_scheduler
from transformers import Trainer, TrainingArguments

In [ ]:
optimizer,_ = get_optimizer(model,config)
scheduler = get_scheduler(optimizer,config)

training_args = TrainingArguments(
  output_dir="./examples", 
  do_train=True,
  remove_unused_columns=False,
  per_device_train_batch_size=1,
  learning_rate=5e-4,
  num_train_epochs=50,
)


trainer = Trainer(optimizers=(optimizer,scheduler),
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"], 
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 194
  Num Epochs = 50
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 850


Step,Training Loss


Saving model checkpoint to ./examples/checkpoint-500
Configuration saved in ./examples/checkpoint-500/config.json


RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 77120 vs 77016